<a href="https://colab.research.google.com/github/Marcin19721205/IntroductionToBigDataWSB/blob/main/Zad11_MJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download latest version
path = kagglehub.dataset_download("shibumohapatra/house-price")

print("Path to dataset files:", path)

### <a name='a0'></a> 1. Import biblotek

In [1]:
#stack import
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

np.set_printoptions(precision=12, suppress=True, linewidth=150)
pd.options.display.float_format = '{:.6f}'.format
tf.__version__

'2.19.0'

In [5]:
#load data
raw_dataset = pd.read_csv('https://storage.googleapis.com/esmartdata-courses-files/ann-course/housing.csv')
raw_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [12]:
#kopia danych surowych
pd.set_option('display.float_format', lambda x: f"{x:.3f}") #wyświetlanie 3 cyfr znaczących
dataset = raw_dataset.copy()
dataset.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.230,37.880,41.000,880.000,129.000,322.000,126.000,8.325,452600.000,NEAR BAY
1,-122.220,37.860,21.000,7099.000,1106.000,2401.000,1138.000,8.301,358500.000,NEAR BAY
2,-122.240,37.850,52.000,1467.000,190.000,496.000,177.000,7.257,352100.000,NEAR BAY
3,-122.250,37.850,52.000,1274.000,235.000,558.000,219.000,5.643,341300.000,NEAR BAY
4,-122.250,37.850,52.000,1627.000,280.000,565.000,259.000,3.846,342200.000,NEAR BAY


In [13]:
dataset.tail(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
20635,-121.090,39.480,25.000,1665.000,374.000,845.000,330.000,1.560,78100.000,INLAND
20636,-121.210,39.490,18.000,697.000,150.000,356.000,114.000,2.557,77100.000,INLAND
20637,-121.220,39.430,17.000,2254.000,485.000,1007.000,433.000,1.700,92300.000,INLAND
20638,-121.320,39.430,18.000,1860.000,409.000,741.000,349.000,1.867,84700.000,INLAND
20639,-121.240,39.370,16.000,2785.000,616.000,1387.000,530.000,2.389,89400.000,INLAND


In [14]:
#sprawdź czy są puste
dataset.isnull().sum() / len(dataset)

,0
longitude,0.000
latitude,0.000
housing_median_age,0.000
total_rooms,0.000
total_bedrooms,0.010
population,0.000
households,0.000
median_income,0.000
median_house_value,0.000
ocean_proximity,0.000


In [16]:
dataset[dataset.isnull().any(axis=1)] #wyświetl wadliwe wiersze

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
290,-122.160,37.770,47.000,1256.000,NaN,570.000,218.000,4.375,161900.000,NEAR BAY
341,-122.170,37.750,38.000,992.000,NaN,732.000,259.000,1.620,85100.000,NEAR BAY
538,-122.280,37.780,29.000,5154.000,NaN,3741.000,1273.000,2.576,173400.000,NEAR BAY
563,-122.240,37.750,45.000,891.000,NaN,384.000,146.000,4.949,247100.000,NEAR BAY
696,-122.100,37.690,41.000,746.000,NaN,387.000,161.000,3.906,178400.000,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20267,-119.190,34.200,18.000,3620.000,NaN,3171.000,779.000,3.341,220500.000,NEAR OCEAN
20268,-119.180,34.190,19.000,2393.000,NaN,1938.000,762.000,1.695,167400.000,NEAR OCEAN
20372,-118.880,34.170,15.000,4260.000,NaN,1701.000,669.000,5.103,410700.000,<1H OCEAN
20460,-118.750,34.290,17.000,5512.000,NaN,2734.000,814.000,6.607,258100.000,<1H OCEAN


Usuniecie rekordów z brakami

In [17]:
dataset.dropna(inplace=True) #usun wadliwe - nie ma sensu średniej itd.

dataset.isnull().sum() / len(dataset)

,0
longitude,0.000
latitude,0.000
housing_median_age,0.000
total_rooms,0.000
total_bedrooms,0.000
population,0.000
households,0.000
median_income,0.000
median_house_value,0.000
ocean_proximity,0.000


In [19]:
dataset.describe().T #wyświetl statystyki w transpozycji - dla danych numerycznych

,count,mean,std,min,25%,50%,75%,max
longitude,20433.000,-119.571,2.004,-124.350,-121.800,-118.490,-118.010,-114.310
latitude,20433.000,35.633,2.136,32.540,33.930,34.260,37.720,41.950
housing_median_age,20433.000,28.633,12.592,1.000,18.000,29.000,37.000,52.000
total_rooms,20433.000,2636.504,2185.270,2.000,1450.000,2127.000,3143.000,39320.000
total_bedrooms,20433.000,537.871,421.385,1.000,296.000,435.000,647.000,6445.000
population,20433.000,1424.947,1133.208,3.000,787.000,1166.000,1722.000,35682.000
households,20433.000,499.433,382.299,1.000,280.000,409.000,604.000,6082.000
median_income,20433.000,3.871,1.899,0.500,2.564,3.537,4.744,15.000
median_house_value,20433.000,206864.413,115435.667,14999.000,119500.000,179700.000,264700.000,500001.000


In [20]:
dataset.describe(include=['object']).T #staystyki dla danych obiektowych

,count,unique,top,freq
ocean_proximity,20433,5,<1H OCEAN,9034


In [21]:
dataset.ocean_proximity.value_counts() #oblicz ilość danych w kolumnie per wartość

,count
ocean_proximity,
<1H OCEAN,9034
INLAND,6496
NEAR OCEAN,2628
NEAR BAY,2270
ISLAND,5


In [23]:
px.histogram(dataset, x='median_house_value') #histogram output (zmiennej docelowej)

In [24]:
dataset.median_house_value.value_counts()

,count
median_house_value,
500001.000,958
137500.000,119
162500.000,116
112500.000,103
187500.000,92
...,...
321700.000,1
300800.000,1
393100.000,1


Wartość 500001 jest niewiarygodna, jest to ucięta do tej wartości cena z inneg przedziału.

odrzucamy te wystąpienia

In [25]:
index_to_drop = dataset[dataset.median_house_value == 500001].index
dataset = dataset.drop(index=index_to_drop)
px.histogram(dataset, x='median_house_value')

In [26]:
dataset_dummies = pd.get_dummies(dataset, drop_first=True) #get dummies - do zakodowania danych kategorycznych, z drop first = True (pomin pierwsze)
dataset_dummies = dataset_dummies.astype({col: float for col in dataset_dummies.select_dtypes('bool').columns}) #bez tego bool są true/false a teraz 0 i 1 - przy standaryzacji nie będzie błedów

dataset_dummies.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-122.230,37.880,41.000,880.000,129.000,322.000,126.000,8.325,452600.000,0.000,0.000,1.000,0.000
1,-122.220,37.860,21.000,7099.000,1106.000,2401.000,1138.000,8.301,358500.000,0.000,0.000,1.000,0.000
2,-122.240,37.850,52.000,1467.000,190.000,496.000,177.000,7.257,352100.000,0.000,0.000,1.000,0.000
3,-122.250,37.850,52.000,1274.000,235.000,558.000,219.000,5.643,341300.000,0.000,0.000,1.000,0.000
4,-122.250,37.850,52.000,1627.000,280.000,565.000,259.000,3.846,342200.000,0.000,0.000,1.000,0.000


In [27]:
#podział na zbiór treningowy i testowy
train_dataset = dataset_dummies.sample(frac=0.8, random_state=0)
test_dataset = dataset_dummies.drop(train_dataset.index)

print(f'train_dataset length: {len(train_dataset)}')
print(f'test_dataset length: {len(test_dataset)}')

train_dataset length: 15580
test_dataset length: 3895


In [28]:
train_dataset.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
19234,-122.690,38.510,18.000,3364.000,501.000,1442.000,506.000,6.685,313000.000,0.000,0.000,0.000,0.000
1859,-124.140,41.950,21.000,2696.000,578.000,1208.000,494.000,2.275,122400.000,0.000,0.000,0.000,1.000
51,-122.270,37.820,43.000,1868.000,456.000,1061.000,407.000,1.504,93800.000,0.000,0.000,1.000,0.000
11192,-117.930,33.820,28.000,2444.000,555.000,1848.000,567.000,3.018,198800.000,0.000,0.000,0.000,0.000
20355,-118.960,34.190,16.000,1807.000,346.000,587.000,296.000,1.981,162500.000,0.000,0.000,0.000,0.000


In [29]:
test_dataset.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
10,-122.260,37.850,52.000,2202.000,434.000,910.000,402.000,3.203,281500.000,0.000,0.000,1.000,0.000
13,-122.260,37.840,52.000,696.000,191.000,345.000,174.000,2.674,191300.000,0.000,0.000,1.000,0.000
19,-122.270,37.840,52.000,1503.000,298.000,690.000,275.000,2.603,162900.000,0.000,0.000,1.000,0.000
26,-122.280,37.850,49.000,1130.000,244.000,607.000,239.000,2.460,93800.000,0.000,0.000,1.000,0.000
40,-122.260,37.830,52.000,1665.000,419.000,946.000,395.000,2.098,155400.000,0.000,0.000,1.000,0.000


In [ ]:
#relacje pomiędzy zmiennymi
px.scatter_matrix(train_dataset, dimensions=['median_house_value', 'housing_median_age', 'median_income', 'total_rooms'], color='median_house_value', height=700)

In [30]:
#wpólczynniki korelacji
cols = ['median_house_value', 'housing_median_age', 'median_income', 'total_rooms']
corr_matrix = train_dataset[cols].corr()

print(corr_matrix)

                    median_house_value  housing_median_age  median_income  \
median_house_value               1.000               0.070          0.639   
housing_median_age               0.070               1.000         -0.196   
median_income                    0.639              -0.196          1.000   
total_rooms                      0.139              -0.369          0.218   

                    total_rooms  
median_house_value        0.139  
housing_median_age       -0.369  
median_income             0.218  
total_rooms               1.000  


In [32]:
corr_matrix.style.background_gradient(cmap='coolwarm').format("{:.2f}") #heatmap korelacji

,median_house_value,housing_median_age,median_income,total_rooms
median_house_value,1.00,0.07,0.64,0.14
housing_median_age,0.07,1.00,-0.20,-0.37
median_income,0.64,-0.20,1.00,0.22
total_rooms,0.14,-0.37,0.22,1.00


In [39]:
#statystyki do standaryzacji
train_stats = train_dataset.describe()
train_stats.pop('median_house_value')
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
longitude,15580.000,-119.559,2.006,-124.350,-121.750,-118.490,-117.990,-114.470
latitude,15580.000,35.649,2.147,32.550,33.930,34.270,37.730,41.950
housing_median_age,15580.000,28.489,12.506,1.000,18.000,29.000,37.000,52.000
total_rooms,15580.000,2620.015,2195.654,2.000,1441.000,2112.000,3119.000,39320.000
total_bedrooms,15580.000,539.901,424.500,2.000,299.000,436.000,647.000,6445.000
population,15580.000,1441.193,1160.543,3.000,801.000,1179.000,1746.000,35682.000
households,15580.000,501.099,385.040,2.000,283.000,411.000,605.000,6082.000
median_income,15580.000,3.672,1.570,0.500,2.520,3.441,4.581,15.000
ocean_proximity_INLAND,15580.000,0.334,0.472,0.000,0.000,0.000,1.000,1.000
ocean_proximity_ISLAND,15580.000,0.000,0.014,0.000,0.000,0.000,0.000,1.000


In [40]:
#Wyrywamy labele - targety z danych - wycinamy labele ze zbioru danych treningowych i testowych
train_labels = train_dataset.pop('median_house_value')
test_labels = test_dataset.pop('median_house_value')

### <a name='a3'></a> 4. Standaryzacja danych
# średnia i Sx są liczone na zbiorze treningowym, ale stosowane na treningowym i testowym

Normalizacja danych

In [41]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

In [42]:
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

In [43]:
normed_train_data.isnull().sum()

,0
longitude,0
latitude,0
housing_median_age,0
total_rooms,0
total_bedrooms,0
population,0
households,0
median_income,0
ocean_proximity_INLAND,0
ocean_proximity_ISLAND,0


In [45]:
normed_train_data.head(3) #dane wystandaryzowane treningowe

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
19234,-1.561,1.333,-0.839,0.339,-0.092,0.001,0.013,1.919,-0.707,-0.014,-0.346,-0.378
1859,-2.283,2.935,-0.599,0.035,0.090,-0.201,-0.018,-0.890,-0.707,-0.014,-0.346,2.644
51,-1.351,1.011,1.160,-0.343,-0.198,-0.328,-0.244,-1.381,-0.707,-0.014,2.889,-0.378


In [47]:
normed_test_data.head(3) #dane wystandaryzowane testowe

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
10,-1.346,1.025,1.880,-0.190,-0.249,-0.458,-0.257,-0.299,-0.707,-0.014,2.889,-0.378
13,-1.346,1.021,1.880,-0.876,-0.822,-0.945,-0.850,-0.636,-0.707,-0.014,2.889,-0.378
19,-1.351,1.021,1.880,-0.509,-0.570,-0.647,-0.587,-0.681,-0.707,-0.014,2.889,-0.378


To wynika z Numpy

In [48]:
normed_test_data = normed_test_data.values
normed_train_data = normed_train_data.values

### <a name='a4'></a> 5. Budowa modelu

In [55]:
#template modelu z regularyzają i dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.regularizers import l2

def build_model():
    model = Sequential()

    # osobna warstwa wejściowa
    model.add(Input(shape=(len(train_dataset.keys()),)))

    model.add(Dense(
        1024,
        activation='relu',
        kernel_regularizer=l2(0.001)
    ))
    model.add(Dropout(0.3))

    model.add(Dense(
        512,
        activation='relu',
        kernel_regularizer=l2(0.001)
    ))
    model.add(Dropout(0.3))

    model.add(Dense(
        128,
        activation='relu',
        kernel_regularizer=l2(0.001)
    ))
    model.add(Dropout(0.3))

    model.add(Dense(1))  # regresja

    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae', 'mse']
    )

    return model


In [56]:
model = build_model()
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 1024)           │        13,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 603,905 (2.30 MB)

 Trainable params: 603,905 (2.30 MB)

 Non-trainable params: 0 (0.00 B)

### <a name='a5'></a> 6. Trenowanie sieci

In [57]:
history = model.fit(normed_train_data, train_labels.values, epochs=150, validation_split=0.2, verbose=1, batch_size=32)

Epoch 1/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - loss: 32106756096.0000 - mae: 146380.9219 - mse: 32106756096.0000 - val_loss: 4636981760.0000 - val_mae: 49635.5430 - val_mse: 4636981760.0000
Epoch 2/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4870359552.0000 - mae: 50558.5820 - mse: 4870359552.0000 - val_loss: 3827284224.0000 - val_mae: 45128.1953 - val_mse: 3827284224.0000
Epoch 3/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 4320668672.0000 - mae: 46985.7500 - mse: 4320668672.0000 - val_loss: 3649613312.0000 - val_mae: 43910.6719 - val_mse: 3649613312.0000
Epoch 4/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 4016859392.0000 - mae: 46187.7305 - mse: 4016859392.0000 - val_loss: 3537404416.0000 - val_mae: 43148.1328 - val_mse: 3537404416.0000
Epoch 5/150
390/390 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4017444864.0000 - mae: 45517.1992 - mse: 4017444864.0000 - val_loss: 3476330240.0000 - val_mae: 42194.5586 - val_mse: 3476330240.0000
Epoch 6/150
390/390 ━

od razu widać, że ładnie wyszła

In [58]:
def plot_hist(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    hist['rmse'] = np.sqrt(hist['mse'])
    hist['val_rmse'] = np.sqrt(hist['val_mse'])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['mae'], name='mae', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_mae'], name='val_mae', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='MAE vs. VAL_MAE', xaxis_title='Epoki', yaxis_title='Mean Absolute Error', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['rmse'], name='rmse', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_rmse'], name='val_rmse', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='RMSE vs. VAL_RMSE', xaxis_title='Epoki', yaxis_title='Root Mean Squared Error', yaxis_type='log')
    fig.show()

plot_hist(history)

Metryka na zbiorze testowym

Porównać należy mae z testowego z mae z treningowego

In [59]:
for name, value in zip(model.metrics_names, model.evaluate(normed_test_data, test_labels.values)):
    print(f'{name:8}{value:.4f}')

122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 2392032000.0000 - mae: 33050.5664 - mse: 2392032000.0000
loss    2595973888.0000
compile_metrics34543.9961


predict tagert na danych testowych

In [60]:
test_predictions = model.predict(normed_test_data).flatten()
test_predictions

122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


array([223098.8  , 179344.2  , 164172.17 , ...,  93727.72 , 119168.555,  66778.34 ], dtype=float32)

In [61]:
pred = pd.DataFrame(test_labels)
pred['predictions'] = test_predictions
pred.head()

,median_house_value,predictions
10,281500.000,223098.797
13,191300.000,179344.203
19,162900.000,164172.172
26,93800.000,145017.859
40,155400.000,161429.156


Wykres Q-Q

In [62]:
#wykres Q-Q jako wpasowanie liniowe
fig = px.scatter(pred, 'median_house_value', 'predictions')
fig.add_trace(go.Scatter(x=[0, 500000], y=[0, 500000], mode='lines'))
fig.show()

In [67]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# --- Dane ---
X = pred['median_house_value'].values.reshape(-1, 1)
y = pred['predictions'].values

# --- Zakres do rysowania ---
x_range = np.linspace(X.min(), X.max(), 500).reshape(-1, 1)

# --- Funkcja do liczenia modeli wielomianowych ---
def fit_poly_regression(X, y, degree):
    poly = PolynomialFeatures(degree=degree)
    X_poly = poly.fit_transform(X)

    model = LinearRegression()
    model.fit(X_poly, y)

    y_pred = model.predict(X_poly)
    IAE = np.sum(np.abs(y - y_pred))
    MSE = mean_squared_error(y, y_pred)

    # predykcja do wykresu
    y_fit = model.predict(poly.transform(x_range))

    # równanie regresji jako tekst
    coefs = model.coef_
    intercept = model.intercept_
    eq = f"y = {intercept:.3f}"
    for i, c in enumerate(coefs[1:], start=1):
        eq += f" + {c:.3f}·x^{i}"

    return y_fit, IAE, MSE, eq

# --- 3 modele ---
y_lin, IAE_lin, MSE_lin, eq_lin = fit_poly_regression(X, y, degree=1)
y_deg2, IAE_deg2, MSE_deg2, eq_deg2 = fit_poly_regression(X, y, degree=2)
y_deg3, IAE_deg3, MSE_deg3, eq_deg3 = fit_poly_regression(X, y, degree=3)

# --- Rysowanie ---
fig = go.Figure()

# punkty
fig.add_trace(go.Scatter(
    x=pred['median_house_value'],
    y=pred['predictions'],
    mode='markers',
    name='Dane'
))

# modele
fig.add_trace(go.Scatter(x=x_range.flatten(), y=y_lin, mode='lines', name='Regresja 1 (liniowa)'))
fig.add_trace(go.Scatter(x=x_range.flatten(), y=y_deg2, mode='lines', name='Regresja 2'))
fig.add_trace(go.Scatter(x=x_range.flatten(), y=y_deg3, mode='lines', name='Regresja 3'))

# linia idealna
fig.add_trace(go.Scatter(
    x=[X.min(), X.max()],
    y=[X.min(), X.max()],
    mode='lines',
    name='Idealna 1:1',
    line=dict(dash='dash')
))

fig.update_layout(
    title="Regresje: rząd 1, 2, 3 oraz linia idealna",
    xaxis_title="median_house_value",
    yaxis_title="predictions"
)

fig.show()

# --- Podsumowanie ---
print("=== PODSUMOWANIE ===\n")

print("▶ RZĄD 1 (LINIOWA)")
print("IAE:", IAE_lin)
print("MSE:", MSE_lin)
print("Regresja:", eq_lin, "\n")

print("▶ RZĄD 2")
print("IAE:", IAE_deg2)
print("MSE:", MSE_deg2)
print("Regresja:", eq_deg2, "\n")

print("▶ RZĄD 3")
print("IAE:", IAE_deg3)
print("MSE:", MSE_deg3)
print("Regresja:", eq_deg3, "\n")


=== PODSUMOWANIE ===

▶ RZĄD 1 (LINIOWA)
IAE: 117068413.15836251
MSE: 1640661133.3495693
Regresja: y = 48805.676 + 0.696·x^1 

▶ RZĄD 2
IAE: 112942358.25584108
MSE: 1566478664.3447258
Regresja: y = 16633.185 + 1.043·x^1 + -0.000·x^2 

▶ RZĄD 3
IAE: 115652931.84452912
MSE: 1609247794.9114766
Regresja: y = 81502.440 + 0.000·x^1 + 0.000·x^2 + -0.000·x^3 



In [68]:
pred.head()

,median_house_value,predictions
10,281500.000,223098.797
13,191300.000,179344.203
19,162900.000,164172.172
26,93800.000,145017.859
40,155400.000,161429.156


In [69]:
pred['error'] = pred['median_house_value'] - pred['predictions']
pred.head()

,median_house_value,predictions,error
10,281500.000,223098.797,58401.203
13,191300.000,179344.203,11955.797
19,162900.000,164172.172,-1272.172
26,93800.000,145017.859,-51217.859
40,155400.000,161429.156,-6029.156


In [ ]:
px.histogram(pred, 'error', marginal='rug', width=1000)